# Sistem Multi Agen : Board Game (Chess)

Anggota:
- Muhammad Wisnu Maulana_162112233074
- Anis Widya Astuti_162112233057
- Nafisatul Nur Ismawati_162112233059
- Rikza Maulana_162112233056 
- Randy Afif_162112233081 


Disini kami menggunakan pettingzoo, library pembelajaran multi agen. Kami memilih pettingzoo karena pettingzoo mirip dengan gymnasium, namun dengan support multi agen.

Untuk framework deeplearningnya sendiri, kami mencoba menggunakan pytorch dan tensorflow. Namun, di akhir nanti akan kami fokuskan di salah satu yang terbaik saja

In [1]:
import torch
import matplotlib.pyplot as plt
from tensordict import TensorDictBase
from tensordict.nn import TensorDictModule, TensorDictSequential
from torch import multiprocessing
from torchrl.collectors import SyncDataCollector
from torchrl.data import LazyMemmapStorage, RandomSampler, ReplayBuffer
from torchrl.modules import AdditiveGaussianWrapper, MultiAgentMLP, ProbabilisticActor, TanhDelta
from torchrl.objectives import DDPGLoss, SoftUpdate, ValueEstimators
from torchrl.record import CSVLogger, PixelRenderTransform, VideoRecorder
from torchrl.envs import check_env_specs, ExplorationType, PettingZooEnv, RewardSum, set_exploration_type, TransformedEnv
import numpy as np
from tqdm import tqdm
import pygame


import tensorflow as tf
from collections import deque
import random

from pettingzoo.classic import chess_v6
from pettingzoo.classic.chess.chess_utils import chess, get_move_plane

# env = chess_v6.env(render_mode="human")
np.set_printoptions(threshold=np.inf)

try:
    is_sphinx = __sphinx_build__
except NameError:
    is_sphinx = False

pygame 2.3.0 (SDL 2.24.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


2024-05-29 00:50:13.902012: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-29 00:50:14.768221: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 00:50:16.265579: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Menggunakan Pytorch (masih error)

In [30]:
seed = 42
torch.manual_seed(seed)

is_fork = multiprocessing.get_start_method() == "fork"
device = "cuda" if torch.cuda.is_available() else "cpu"

frames_per_batch = 1000
n_iters = 5
total_frames = frames_per_batch * n_iters

iteration_when_stop_training_evaders = n_iters//2

memory_size = 1000000
n_optimiser_steps = 100
train_batch_size = 128
lr = 1e-3
max_grad_norm = 1.0

gamma = 0.99
polyak_tau = 0.005

In [31]:
max_steps = 100

n_chasers = 2
n_evaders = 1
n_obstacles = 2

base_env = PettingZooEnv(
    task = "chess_v6",
    parallel = False,
    seed = seed,
    use_mask = True
)

print("action_spec:", base_env.full_action_spec)
print("reward_spec:", base_env.full_reward_spec)
print("done_spec:", base_env.full_done_spec)
print("observation_spec:", base_env.observation_spec)

action_spec: CompositeSpec(
    player_0: CompositeSpec(
        action: DiscreteTensorSpec(
            shape=torch.Size([1]),
            space=DiscreteBox(n=4672),
            device=cpu,
            dtype=torch.int64,
            domain=discrete), device=cpu, shape=torch.Size([1])),
    player_1: CompositeSpec(
        action: DiscreteTensorSpec(
            shape=torch.Size([1]),
            space=DiscreteBox(n=4672),
            device=cpu,
            dtype=torch.int64,
            domain=discrete), device=cpu, shape=torch.Size([1])), device=cpu, shape=torch.Size([]))
reward_spec: CompositeSpec(
    player_0: CompositeSpec(
        reward: UnboundedContinuousTensorSpec(
            shape=torch.Size([1, 1]),
            space=None,
            device=cpu,
            dtype=torch.float32,
            domain=continuous), device=cpu, shape=torch.Size([1])),
    player_1: CompositeSpec(
        reward: UnboundedContinuousTensorSpec(
            shape=torch.Size([1, 1]),
            s

/home/xmaulana/.pyenv/versions/boardgame/lib/python3.10/site-packages/torchrl/envs/common.py:2989: DeprecationWarning: Your wrapper was not given a device. Currently, this value will default to 'cpu'. From v0.5 it will default to `None`. With a device of None, no device casting is performed and the resulting tensordicts are deviceless. Please set your device accordingly.
  warnings.warn(
/home/xmaulana/.pyenv/versions/boardgame/lib/python3.10/site-packages/torchrl/envs/libs/pettingzoo.py:1005: UserWarning: PettingZoo failed to load all modules with error message No module named 'multi_agent_ale_py', trying to load individual modules.
  warnings.warn(
/home/xmaulana/.pyenv/versions/boardgame/lib/python3.10/site-packages/torchrl/envs/libs/pettingzoo.py:56: UserWarning: SISL environments failed to load with error message No module named 'pymunk'.
  warnings.warn(f"SISL environments failed to load with error message {err}.")
/home/xmaulana/.pyenv/versions/boardgame/lib/python3.10/site-pack

In [32]:
print("action_keys:", base_env.action_keys)
print("reward_keys:", base_env.reward_keys)
print("done_keys:", base_env.done_keys)

action_keys: [('player_0', 'action'), ('player_1', 'action')]
reward_keys: [('player_0', 'reward'), ('player_1', 'reward')]
done_keys: ['done', 'terminated', 'truncated', ('player_0', 'done'), ('player_0', 'terminated'), ('player_0', 'truncated'), ('player_1', 'done'), ('player_1', 'terminated'), ('player_1', 'truncated')]


In [33]:
env = TransformedEnv(
    base_env,
    RewardSum(
        in_keys=base_env.reward_keys,
        reset_keys=["_reset"] * len(base_env.group_map.keys()),
    ),
)

In [34]:
check_env_specs(env)

2024-05-24 03:45:28,014 [torchrl][INFO] check_env_specs succeeded!


[WARNING]: Illegal move made, game terminating with current player losing. 
obs['action_mask'] contains a mask of all legal moves that can be chosen.


In [35]:
n_rollout_steps = 5
rollout = env.rollout(n_rollout_steps)
print(f"rollout of {n_rollout_steps} steps:", rollout)
print("Shape of the rollout TensorDict:", rollout.batch_size)

[WARNING]: Illegal move made, game terminating with current player losing. 
obs['action_mask'] contains a mask of all legal moves that can be chosen.
rollout of 5 steps: TensorDict(
    fields={
        done: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                player_0: TensorDict(
                    fields={
                        action_mask: Tensor(shape=torch.Size([1, 1, 4672]), device=cpu, dtype=torch.bool, is_shared=False),
                        done: Tensor(shape=torch.Size([1, 1, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                        episode_reward: Tensor(shape=torch.Size([1, 1, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                        mask: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                 

In [36]:
policy_modules = {}
for group, agents in env.group_map.items():
    share_parameters_policy = True  # Can change this based on the group

    policy_net = MultiAgentMLP(
        n_agent_inputs=env.observation_spec[group, "observation"].shape[
            -1
        ],
        n_agent_outputs=env.full_action_spec[group, "action"].shape[
            -1
        ],
        n_agents=len(agents),
        centralised=False, 
        share_params=share_parameters_policy,
        device=device,
        depth=2,
        num_cells=256,
        activation_class=torch.nn.Tanh,
    )
    policy_module = TensorDictModule(
        policy_net,
        in_keys=[(group, "observation")],
        out_keys=[(group, "param")],
    ) 
    policy_modules[group] = policy_module

### Menggunakan Tensorflow (Masih saya uji coba)

In [2]:
#Hyperparameter
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 1e-3
batch_size = 8
memory_size = 2000

In [3]:
#Neural net
class QNetwork(tf.keras.Model):
    def __init__(self, action_size):
        super(QNetwork, self).__init__()
        # self.input_layer = tf.keras.Input(shape=(action_size,))
        self.dense1 = tf.keras.layers.Dense(64, input_shape=(action_size,), activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.out = tf.keras.layers.Dense(action_size, activation='softmax')
        
    def call(self, state):
        # x = self.input_layer(state)
        x = self.dense1(state)
        x = self.dense2(x)
        return self.out(x)

class ReplayMemory:
    def __init__(self, size):
        self.memory = deque(maxlen=size)
        
    def add(self, experience):
        self.memory.append(experience)
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
        
    def __len__(self):
        return len(self.memory)

class DQNAgent:
    def __init__(self, action_size):
        self.action_size = action_size
        self.memory = ReplayMemory(memory_size)
        self.model = QNetwork(action_size)
        self.target_model = QNetwork(action_size)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        self.update_target_network()
        
    def update_target_network(self):
        self.target_model.set_weights(self.model.get_weights())
        
    def act(self, state):
        if np.random.rand() <= epsilon:
            available = [i for i,x in enumerate(state) if x[0]]
            # print(np.array(state[0][0]["action_mask"]))
            randome = np.random.choice(available)
            return randome
        q_values = self.model(state)
        print(q_values)
        return np.argmax(q_values[0])
    
    def remember(self, state, action, reward, truncation):
        self.memory.add((state, action, reward, truncation))
        
    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = self.memory.sample(batch_size)
        
        for state, action, reward, truncation in minibatch:
            target = self.model(state).numpy()
            if truncation:
                target[0][action] = reward
            else:
                t = self.target_model(state).numpy()
                target[0][action] = reward + gamma * np.amax(t[0])
                
            with tf.GradientTape() as tape:
                q_values = self.model(state)
                loss = tf.keras.losses.MSE(target, q_values)
            grads = tape.gradient(loss, self.model.trainable_variables)
            self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
        
        global epsilon
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

In [ ]:
env = None
is_raw = True

if not is_raw:
    env = chess_v6.env(render_mode="human")
else:
    env = chess_v6.raw_env()
env.reset(seed=42)
episodes = 100
running = True
agents = {agent: DQNAgent(env.action_space(agent).n) for agent in env.agents}

for i in range(episodes):
    print(f"EPISODE : {i}")
    env.reset(seed=42)
    
    for agent in env.agent_iter():
        observation, reward, termination, truncation, info = env.last()

        if not is_raw:
            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        termination = True
                        running = False
                        break
                    
        if termination or truncation:
            action = None
            break
        else:
            mask = observation["action_mask"]
            mask = np.expand_dims(mask, axis=1)
            
            # available = [i for i,x in enumerate(mask) if x]
            # print(available)
            # action = np.random.choice(available)
            
            action = agents[agent].act(mask)
            agents[agent].remember(mask, action, reward, truncation)
            # if episodes > 0:
            #     print(action)
        
        env.step(action)
    for agent in agents:
        agents[agent].replay(batch_size)
    
    if not running:
        break
env.close()

EPISODE : 0
EPISODE : 1


### Test dengan random move

In [ ]:
env = chess_v6.env(render_mode="human")

episodes = 100
running = True
for i in range(episodes):
    print(f"EPISODE : {i}")
    env.reset(seed=42)
    for agent in env.agent_iter():
        observation, reward, termination, truncation, info = env.last()
        
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    termination = True
                    running = False
                    break
                    
        if termination or truncation:
            action = None
            break
        else:
            mask = observation["action_mask"]
            
            available = [i for i,x in enumerate(mask) if x]
            print(available)
            action = np.random.choice(available)
            # action = int(input("Move: "))
            # print(action)
    
        env.step(action)
    if not running:
        break
env.close()

In [3]:
move = chess.Move(from_square=8*2+4, to_square=8*2+5, promotion=None)
get_move_plane(move)

6

In [11]:
get_move_plane(move)

6

In [4]:
!pip install python-chess

In [5]:
import chess

In [7]:
board  = chess.Board()

In [16]:
board

AttributeError: 'Board' object has no attribute 'can_claim_threeflod_repetition'

In [14]:
board.push_san("Nf6")

Move.from_uci('g8f6')

In [13]:
board.legal_moves

<LegalMoveGenerator at 0x7f57a577fcd0 (Nh6, Nf6, Nc6, Na6, h6, g6, f6, e6, d6, c6, b6, a6, h5, g5, f5, e5, d5, c5, b5, a5)>

In [18]:
board.can_claim_threefold_repetition()

False